In [7]:
import numpy as np
import os
import json
import pandas as pd

from utils.evaluation import *
data_path="/home/emartini/nas/MAEVE/dataset/panoptic-toolbox/trtpose3D/"
sequence_name = "161029_sports1" #161029_sports1

CONTINUOUS_STATE_PARTS = [
            "nose", "left_ear", "right_ear", "left_shoulder", "right_shoulder", 
            "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", 
            "right_knee", "left_ankle", "right_ankle", "neck"]

# Load the json for comparison

mapping = [12, 7, 10, 4,  5, 9, 6, 8, 11, 3, 14, 13]
AP_val = 0.5
header = ["frame_id"]+[CONTINUOUS_STATE_PARTS[m] for m in mapping]

In [8]:
# Load ground truth
GT = {}
with open(os.path.join(data_path,sequence_name+".gt.json"), "r") as f:
    ground_truth = json.load(f)
    for frame in ground_truth:
        GT[frame["timestamp"]] = frame        

In [9]:
# # Baseline (compute HOTA, MOTA, and MOTP)
# cameras = [6,7,8,9,10] # average the value across all cameras
# # cameras = [6] # average the value across all cameras
# cameras_results = []
# for camera in cameras:
#     print("CAM",camera)
#     # Load camera file
#     DUT = {}
#     with open(os.path.join(data_path,sequence_name+"."+ str(camera) +".json"), "r") as f:
#         file = json.load(f)
#     for frame in file:
#         DUT[frame["timestamp"]] = frame    
    
#     ## Build the triple nested list (shape: n_frames, n_people, n_joints, 3) and IDs (shape: n_frames, n_people)
#     # Set unique ids (union of both gt and camera)
#     ids = list(GT.keys())
#     for frame_dut in list(DUT.keys()):
#         if frame_dut not in ids:
#             ids.append(frame_dut) 
            
#     predicted_keypoints = []
#     predicted_ids = []
#     ground_truth_keypoints = []
#     ground_truth_ids = []
#     for id in sorted(ids):
#         predicted_keypoints_per_frame = []
#         predicted_ids_per_frame = []
#         ground_truth_keypoints_per_frame = []
#         ground_truth_ids_per_frame = []
#         if id in GT.keys():
#             for pp in GT[id]['continuousState']:
#                 s = np.array([ [np.nan,np.nan,np.nan] if not f else f for f in pp])
#                 s = s[mapping,:]
#                 ground_truth_keypoints_per_frame.append(s)
#             for i in GT[id]['track_ids']:
#                 ground_truth_ids_per_frame.append(i)
#         if id in DUT.keys():
#             for i,pp in enumerate(DUT[id]['continuousState']):
#                 s = np.array([ [np.nan,np.nan,np.nan] if not f else f for f in pp])
#                 s = s[mapping,:]
#                 if not np.isnan(s).all():
#                     predicted_keypoints_per_frame.append(s)
#                     predicted_ids_per_frame.append(i)

#             # This doesn't work for cameras
#             # for id in DUT[id]['track_ids']:
#             #     predicted_ids_per_frame.append(id)
            
#         predicted_keypoints.append(predicted_keypoints_per_frame)
#         predicted_ids.append(predicted_ids_per_frame)
#         ground_truth_keypoints.append(ground_truth_keypoints_per_frame)
#         ground_truth_ids.append(ground_truth_ids_per_frame)
    
#     hota = hota_3d(predicted_keypoints, predicted_ids, ground_truth_keypoints, ground_truth_ids,distance_threshold=AP_val)
#     mota,motp = mota_motp(predicted_keypoints, predicted_ids, ground_truth_keypoints, ground_truth_ids,distance_threshold=AP_val)
#     print("HOTA:",hota,'\t',"MOTA:",round(mota,1),'\t',"MOTP:",round(motp,2))
#     cameras_results.append([hota,mota,motp])
# print(np.mean(np.array(cameras_results),axis=0))

In [10]:
# match = [[a,b] for a,b in zip(predicted_ids,ground_truth_ids)]
# for m in match:
#     print(m)


In [24]:
# Compared methods (HOTA, MOTA, and MOTP)
# cameras = [6,7,8,9,10] # average the value across all cameras
cameras = [6,7,8,9,10] # average the value across all cameras
methods = ["befine","cometh"]
sequences = ["160422_ultimatum1","160906_band4","161029_build1","161029_sports1","161029_tools1","170407_haggling_a1","170407_office2"]
result = []

for sequence_name in sequences:
    for method in methods:
        # Load ground truth
        GT = {}
        with open(os.path.join(data_path,sequence_name+".gt.json"), "r") as f:
            ground_truth = json.load(f)
            for frame in ground_truth:
                GT[frame["timestamp"]] = frame        


        # Load camera file
        DUT = {}
        with open(os.path.join(data_path,sequence_name+"."+ method+"." +".".join(map(str, cameras)) +".json" ), "r") as f:
            file = json.load(f)
        for frame in file:
            DUT[frame["timestamp"]] = frame    

        ## Build the triple nested list (shape: n_frames, n_people, n_joints, 3) and IDs (shape: n_frames, n_people)
        # Set unique ids (union of both gt and camera)
        ids = list(GT.keys())
        for frame_dut in list(DUT.keys()):
            if frame_dut not in ids:
                ids.append(frame_dut) 
                
        predicted_keypoints = []
        predicted_ids = []
        ground_truth_keypoints = []
        ground_truth_ids = []
        for id in sorted(ids):
            predicted_keypoints_per_frame = []
            predicted_ids_per_frame = []
            ground_truth_keypoints_per_frame = []
            ground_truth_ids_per_frame = []
            if id in GT.keys():
                for pp in GT[id]['continuousState']:
                    s = np.array([ [np.nan,np.nan,np.nan] if not f else f for f in pp])
                    s = s[mapping,:]
                    ground_truth_keypoints_per_frame.append(s)
                for i in GT[id]['track_ids']:
                    ground_truth_ids_per_frame.append(i)
            if id in DUT.keys():
                for i,pp in enumerate(DUT[id]['continuousState']):
                    s = np.array([ [np.nan,np.nan,np.nan] if not f else f for f in pp])
                    s = s[mapping,:]
                    if not np.isnan(s).all():
                        predicted_keypoints_per_frame.append(s)
                        # predicted_ids_per_frame.append(i)

                # This doesn't work for cameras
                for I in DUT[id]['track_ids']:
                    predicted_ids_per_frame.append(I)
            # print(id)
            # if id > 500:
            #     # print(GT[id]['continuousState'])
            #     # print(predicted_keypoints_per_frame)
            #     # print(predicted_ids_per_frame)
            #     # print(ground_truth_keypoints_per_frame)
            #     # print(ground_truth_ids_per_frame)
            
            #     break
            
            predicted_keypoints.append(predicted_keypoints_per_frame)
            predicted_ids.append(predicted_ids_per_frame)
            ground_truth_keypoints.append(ground_truth_keypoints_per_frame)
            ground_truth_ids.append(ground_truth_ids_per_frame)

        # print(predicted_keypoints)

        # Integral
        step = 0.05
        res = []
        while step < 1:
            print(step)
            loca, deta, detpr, detre, assa,asspr,assre, hot = hota(predicted_keypoints, predicted_ids, ground_truth_keypoints, ground_truth_ids,distance_threshold=step)
            res.append([step, loca, deta, detpr, detre, assa,asspr,assre, hot])
            step += 0.05

        row = [sequence_name,method] + list(np.nanmean(np.array(res),0)[1:])
        result.append(row)

0.1
0.15000000000000002
0.2
0.25
0.3
0.35
0.39999999999999997
0.44999999999999996
0.49999999999999994
0.5499999999999999
0.6
0.65
0.7000000000000001
0.7500000000000001
0.8000000000000002
0.8500000000000002
0.9000000000000002
0.9500000000000003
0.1
0.15000000000000002
0.2
0.25
0.3
0.35
0.39999999999999997
0.44999999999999996
0.49999999999999994
0.5499999999999999
0.6
0.65
0.7000000000000001
0.7500000000000001
0.8000000000000002
0.8500000000000002
0.9000000000000002
0.9500000000000003


In [25]:
print(result)
header_stats = ["Aggregator", "Sequence", "LocA", "DetA", "DetPR", "DetRE", "AssA","AssPR","AssRE", "HOTA"]
pd.DataFrame(result,columns=header_stats)

[['161029_sports1', 'befine', 0.8679008491194675, 0.6022711344785483, 0.8396567299006323, 0.6644604671226967, 0.1416522352153656, 0.15411223488311251, 0.6251348873121978, 0.292014737010659], ['161029_sports1', 'cometh', 0.8562845826320147, 0.5746199260176682, 0.7918563950066599, 0.6510173937402909, 0.5679526554311572, 0.5920601247980062, 0.9028283589389849, 0.5705292627150668]]


,Aggregator,Sequence,LocA,DetA,DetPR,DetRE,AssA,AssPR,AssRE,HOTA
0,161029_sports1,befine,0.867901,0.602271,0.839657,0.664460,0.141652,0.154112,0.625135,0.292015
1,161029_sports1,cometh,0.856285,0.574620,0.791856,0.651017,0.567953,0.592060,0.902828,0.570529


In [ ]:
# Build qualitative json method vs ground truth (matching based on frame_id)
import collections
result = collections.Counter(list(DUT.keys())) & collections.Counter(list(GT.keys()))
intersected_list = list(result.elements())
# print(DUT.keys())
out = []
for i in intersected_list:
    f_gt = GT[i]
    f_dut = DUT[i]
    summed = f_gt.copy()
    summed["kp3d"] += f_dut["kp3d"]
    summed["kp2d"] += f_dut["kp2d"]
    summed["continuousState"] += f_dut["continuousState"]
    summed["track_ids"] += [id + 50 for id in f_dut["track_ids"]]
    # print(summed)
    # print(f_gt["timestamp"],f_dut["timestamp"])
    out.append(summed)
    # break

# dump json
filename = os.path.join(data_path,sequence_name+".gt_vs_"+method+"." +".".join(map(str, cameras))+".json")
print("Written",filename)

with open(filename, "w") as outfile:
    json.dump(out, outfile)

# Export in viewer format
os.system('python3 to_viewer.py ' + filename + ' --rotation 180 --output '+filename.replace('.json','.viewer.json')+' >> "tmp/"$1"_log_2.txt"')

In [ ]:
# Build qualitative json method vs ground truth (matching based on timestamps)
GT_t = {}
for frame in list(GT.keys()):
    GT_t[GT[frame]["timestamp"]] = GT[frame]

DUT_t = {}
for frame in list(DUT.keys()):
    DUT_t[DUT[frame]["timestamp"]] = DUT[frame]
          
import collections
result = collections.Counter(list(DUT_t.keys())) & collections.Counter(list(DUT_t.keys()))
intersected_list = list(result.elements())
# print(DUT.keys())
intersected_list.remove(0)
out = []
for i in intersected_list:
    print(i)
    summed = {}
    f_gt = GT_t[i]
    f_dut = DUT_t[i]
    # summed = f_gt.copy()
    summed["timestamp"] = f_gt["timestamp"]
    summed["frame_id"] = f_gt["frame_id"]
    summed["kp3d"] = f_gt["kp3d"]+f_dut["kp3d"]
    summed["kp2d"] = f_gt["kp2d"]+f_dut["kp2d"]
    summed["continuousState"] = f_gt["continuousState"]+f_dut["continuousState"]
    summed["track_ids"] = f_gt["track_ids"]+ [id + 50 for id in f_dut["track_ids"]]

    out.append(summed)

# dump json
filename = os.path.join(data_path,sequence_name+".gt_vs_"+method+"." +".".join(map(str, cameras))+".json")

with open(filename, "w") as outfile:
    json.dump(out, outfile)

# Export in viewer format
os.system('python3 to_viewer.py ' + filename + ' --rotation 180 --output '+filename.replace('.json','.viewer.json')+' >> "tmp/"$1"_log_2.txt"')